## LIME と SHAP によるデータセット分析

In [77]:
import pandas as pd
import python.data_manager as dm
import importlib

In [78]:
importlib.reload(dm)

2024-05-28 21:14:36,506 : python.data_manager : INFO : 32 : Test_message


<module 'python.data_manager' from 'c:\\Users\\zigza\\OneDrive\\ドキュメント\\0_大学院\\Projects\\Test_LIME_SHAP\\src\\python\\data_manager.py'>

In [79]:
left_df  = dm.read_folder("../data/Reflection/")
right_df = dm.read_folder("../data/Grade/")

2024-05-28 21:14:36,522 : python.data_manager : INFO : 77 : read ..\data\Reflection *.csv data...
2024-05-28 21:14:36,524 : python.data_manager : INFO : 80 : read : [WindowsPath('../data/Reflection/Course_C-2021-1_Reflection.csv'), WindowsPath('../data/Reflection/Course_C-2021-2_Reflection.csv'), WindowsPath('../data/Reflection/Course_C-2022-1_Reflection.csv')]
2024-05-28 21:14:36,676 : python.data_manager : INFO : 87 : get 28275 rows.
2024-05-28 21:14:36,677 : python.data_manager : INFO : 77 : read ..\data\Grade *.csv data...
2024-05-28 21:14:36,678 : python.data_manager : INFO : 80 : read : [WindowsPath('../data/Grade/Course_C-2021-1_GradePoint.csv'), WindowsPath('../data/Grade/Course_C-2021-2_GradePoint.csv'), WindowsPath('../data/Grade/Course_C-2022-1_GradePoint.csv')]
2024-05-28 21:14:36,686 : python.data_manager : INFO : 87 : get 378 rows.


In [80]:
left_df.head()

,userid,course_number,question_number,question_content,answer_content
0,C-2021-1_U57,1,1,今日の内容を自分なりの言葉で説明してみてください,携帯やpcは音声、文章、写真を伝える。現在の通信技術に至るまでに、さまざまな技術が生み出され...
1,C-2021-1_U57,1,2,今日の内容で、分かったこと・できたことを書いてください,情報は１と０で表され、その組み合わせで伝えられる。
2,C-2021-1_U57,1,3,今日の内容で、分からなかったこと・できなかったことを書いてください,特になし。
3,C-2021-1_U57,1,4,質問があれば書いてください,特になし。
4,C-2021-1_U57,1,5,今日の授業の感想や反省を書いてください,特になし。


In [81]:
key = "userid"
df = pd.merge(left=left_df, right=right_df, on=key)

In [82]:
df.head()

,userid,course_number,question_number,question_content,answer_content,grade
0,C-2021-1_U57,1,1,今日の内容を自分なりの言葉で説明してみてください,携帯やpcは音声、文章、写真を伝える。現在の通信技術に至るまでに、さまざまな技術が生み出され...,B
1,C-2021-1_U57,1,2,今日の内容で、分かったこと・できたことを書いてください,情報は１と０で表され、その組み合わせで伝えられる。,B
2,C-2021-1_U57,1,3,今日の内容で、分からなかったこと・できなかったことを書いてください,特になし。,B
3,C-2021-1_U57,1,4,質問があれば書いてください,特になし。,B
4,C-2021-1_U57,1,5,今日の授業の感想や反省を書いてください,特になし。,B


In [83]:
LABEL_MAPPING = {"A": 0, "B": 1, "C": 2, "D": 3, "F": 4}
df["label"] = df["grade"].map(LABEL_MAPPING).astype(int)

In [85]:
_, dump = dm.data_clean(df)
dump.head()

2024-05-28 21:14:36,842 : python.data_manager : INFO : 221 : df shape : (28275, 7)
2024-05-28 21:14:36,843 : python.data_manager : INFO : 222 : cleaned_df shape : (20291, 7)
2024-05-28 21:14:36,844 : python.data_manager : INFO : 223 : dumped 7984 rows.
2024-05-28 21:14:36,855 : python.data_manager : INFO : 231 : dump_df shape : (7984, 7)


,userid,course_number,question_number,question_content,answer_content,grade,label
18,C-2021-1_U57,4,4,質問があれば書いてください,NaN,B,1
28,C-2021-1_U57,6,4,質問があれば書いてください,NaN,B,1
33,C-2021-1_U57,7,4,質問があれば書いてください,NaN,B,1
42,C-2021-1_U57,9,3,今日の内容で、分からなかったこと・できなかったことを書いてください,NaN,B,1
43,C-2021-1_U57,9,4,質問があれば書いてください,NaN,B,1


In [86]:
train, valid, test = dm.sprit_data_for_user(df, key=key, text="answer_content", label="grade", split_rate=[0.8, 0.1, 0.1])

In [87]:
print(f"""
      columns =
      train : {train.columns}
      valid : {valid.columns}
      test  : {test.columns}
      """
)


      columns =
      train : Index(['userid', 'grade', 'text', 'course_number', 'question_number', 'label'], dtype='object')
      valid : Index(['userid', 'grade', 'text', 'course_number', 'question_number', 'label'], dtype='object')
      test  : Index(['userid', 'grade', 'text', 'course_number', 'question_number', 'label'], dtype='object')
      


In [88]:
print(f"""
      rows =
      train : {len(train)}\t- {len(train) / (len(train) + len(valid) + len(test))} %
      valid : {len(valid)}\t- {len(valid) / (len(train) + len(valid) + len(test))} %
      test  : {len(test)}\t- {len(test ) / (len(train) + len(valid) + len(test))} %
      """
)


      rows =
      train : 22575	- 0.7984084880636605 %
      valid : 2850	- 0.10079575596816977 %
      test  : 2850	- 0.10079575596816977 %
      


In [89]:
train.head()

,userid,grade,text,course_number,question_number,label
0,C-2022-1_U80,C,情報伝達の歴史について\r\n〇or✖のようなシンプルな情報伝達から取り決めによってより具体...,01,1,2
1,C-2021-1_U81,D,イントロダクション 情報通信方法,01,1,3
2,C-2021-2_U2,F,nan,01,1,4
3,C-2021-2_U54,B,外部Moodleの使い方や、リアルタイムモニタリングの閲覧の仕方を学んだ。人間ははじめは言葉...,01,1,1
4,C-2021-2_U98,C,今回は新しいmoodleのシステム扱い方や通信技術の起源を簡単に触れた,01,1,2


In [ ]:
text = "answer_content"
label = "label"
categories = train[label].unique().tolist()

### 分類機実装

In [90]:
import MeCab
import re

tagger = MeCab.Tagger("-Owakati")


def make_wakati(sentence):
    sentence = tagger.parse(sentence)
    sentence = re.sub(r"[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+", " ", sentence)
    sentence = re.sub(
        r"[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=＝)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+",
        "",
        sentence,
    )
    wakati = sentence.split(" ")
    wakati = list(filter(("").__ne__, wakati))
    return wakati

ModuleNotFoundError: No module named 'MeCab'

In [ ]:
from gensim.models import Word2Vec
import numpy as np
import logging
import tqdm

# word2vec parameters
num_features = 200
min_word_count = 5
num_workers = 40
context = 10
downsampling = 1e-3
model_name = "word2vec_testcase.model"

# コーパス読み込み
corpus = []
for doc in tqdm(df["answer_content"]):
    corpus.append(make_wakati(doc))


# word2vecモデルの作成＆モデルの保存
print("cleating word2vec model ...")
logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)
model = Word2Vec(
    corpus,
    workers=num_workers,
    hs=0,
    sg=1,
    negative=10,
    iter=25,
    size=num_features,
    min_count=min_word_count,
    window=context,
    sample=downsampling,
    seed=1,
)
model.save(model_name)
print("Done.")

# word2vecモデルの読み込み
# model = Word2Vec.load(model_name)

# test
# print(model.wv.most_similar("フーリエ変換"))

In [ ]:
def wordvec2docvec(sentence):
    # 文章ベクトルの初期値（0ベクトルを初期値とする）
    docvecs = np.zeros(num_features, dtype="float32")

    # 文章に現れる単語のうち、モデルに存在しない単語をカウントする
    denomenator = len(sentence)

    # 文章内の各単語ベクトルを足し合わせる
    for word in sentence:
        try:
            temp = model[word]
        except:
            denomenator -= 1
            continue
        docvecs += temp

    # 文章に現れる単語のうち、モデルに存在した単語の数で割る
    if denomenator > 0:
        docvecs = docvecs / denomenator

    return docvecs

In [ ]:
from sklearn.model_selection import train_test_split

print(len(df[text]))
X, Y = [], []
for doc, category in tqdm(zip(df[text], df[label])):
    wakati = make_wakati(doc)
    docvec = wordvec2docvec(wakati)
    X.append(list(docvec))
    Y.append(LABEL_MAPPING[category])
data_X = pd.DataFrame(X, columns=["X" + str(i + 1) for i in range(num_features)])
data_Y = pd.DataFrame(Y, columns=["category_id"])

train_x, test_x, train_y, test_y = train_test_split(data_X, data_Y, train_size=0.7)

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report

print("Fitting XGboost model ...")
xgb_model = xgb.XGBClassifier()
xgb_model.fit(train_x, train_y)
print("Done.")

# 予測
pred = xgb_model.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))

In [ ]:
import lightgbm as lgbm

print("Fitting LightGBM model ...")
lgbm_model = lgbm.LGBMClassifier()
lgbm_model.fit(train_x, train_y)
print("Done.")

# 予測
pred = lgbm_model.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(train_x, train_y)
pred = rfc.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = datasets["document"]
tfidf_vectorizer = TfidfVectorizer(analyzer=make_wakati)
tfidfs = tfidf_vectorizer.fit_transform(corpus)
print(tfidfs.shape)
# (7376, 79673)

tfidf_data_X = pd.DataFrame(
    tfidfs.toarray(), columns=["X" + str(i) for i in range(tfidfs.shape[1])]
)
train_x, test_x, train_y, test_y = train_test_split(data_X, data_Y, train_size=0.7)
lgbm_model = lgbm.LGBMClassifier()
lgbm_model.fit(train_x, train_y)
pred = lgbm_model.predict(test_x)
print(classification_report(pred, test_y["category_id"], target_names=categories))